In [268]:
%reset
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np
import defmod as dm
import multimodule_usefulfunctions as mm 
import matplotlib.pyplot as plt

torch.set_default_tensor_type(torch.DoubleTensor)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [269]:
nb_pts1 = 10
nb_pts2 = 7
dim = 2

o1 = torch.tensor([1.,1.])#.float()
o2 = torch.tensor([5.,5.])#.float()
origin = [o1,o2]
radius = [2.,1.]
numberPoints = [nb_pts1, nb_pts2]
gd_list = mm.multipleCircles(origin, radius, numberPoints)

In [270]:

nb_pts1 = 10
gd_list = mm.multipleCircles([o1], [radius[1]], [nb_pts1])

In [271]:
# Defining the deformation modules
sigma1 = 10
manifold1 = dm.manifold.Landmarks(dim, nb_pts1, gd=gd_list[0].view(-1))
trans1 = dm.deformationmodules.Translations(manifold1, sigma1)

sigma2 = 10
#manifold2 = dm.manifold.Landmarks(dim, nb_pts2, gd=gd_list[1].view(-1))
#trans2 = dm.deformationmodules.Translations(manifold2, sigma2)

#module_list = [trans1, trans2]
sigma_bg = 1
#modules = dm.multishape.MultiShapeModule(module_list, sigma_bg)
modules = dm.multishape.MultiShapeModule([trans1], sigma_bg)

In [272]:
# Setting cotangent variables
p1 = torch.cat([torch.ones(nb_pts1,1, requires_grad=True), torch.zeros(nb_pts1,1)],1).view(-1)
p2 = torch.cat([-torch.ones(nb_pts2,1,requires_grad=True), torch.zeros(nb_pts2,1)],1).view(-1)
#cotan2 = [p1, p2, [p1, p2]]
#cotan = [torch.rand(nb_pts1*dim, requires_grad=True), -torch.rand(nb_pts2*dim, requires_grad=True), [torch.rand((nb_pts1)*dim, requires_grad=True),-torch.rand((nb_pts2)*dim, requires_grad=True)]]

p1 = torch.ones(nb_pts1,dim, requires_grad=True).view(-1)
cotan = [p1, [p1]]
modules.manifold.fill_cotan(cotan)



In [273]:
constr_Id = dm.constraints.Identity()
l, h = modules.compute_geodesic_variables(modules.manifold, constr_Id)

____________


In [274]:
import defmod.hamiltonian_multishape as hamiltonian
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

In [275]:
def controlslist2tensor(controls_list):
    return torch.cat([modules.controls[0].view(-1,1),modules.controls[1].view(-1,1), modules.controls[-1][0].view(-1,1),modules.controls[-1][1].view(-1,1)])

def controlstensor2list(controls, nb_pts, dim):
    n = sum(nb_pts)
    return [controls[:dim*nb_pts[0]], controls[dim*nb_pts[0]:dim*(nb_pts[0]+nb_pts[1])], [controls[dim*n:dim*(n+nb_pts[0])], controls[dim*(n+nb_pts[0]):]]]

def gdtensor2list(gdtensor, nb_pts, dim):
    gdlist = []
    gdlist_bg = []
    j = 0
    n = sum(nb_pts)
    for i in range(len(nb_pts)):
        gdlist.append(gdtensor[dim*j:dim*(j+nb_pts[i])])
        gdlist_bg.append(gdtensor[dim*(n+j):dim*(n+j+nb_pts[i])])
        j = j + nb_pts[i]
    return [*gdlist, gdlist_bg]

def gdlist2tensor(gdlist):
    return torch.cat([*gdlist[:-1],*gdlist[-1]],0)



In [276]:
def controlslist2tensor(controls_list):
    return torch.cat([modules.controls[0].view(-1,1), modules.controls[-1][0].view(-1,1)])

def controlstensor2list(controls, nb_pts, dim):
    n = sum(nb_pts)
    return [controls[:dim*nb_pts[0]], [controls[dim*n:dim*(n+nb_pts[0])]]]

def gdtensor2list(gdtensor, nb_pts, dim):
    gdlist = []
    gdlist_bg = []
    j = 0
    n = sum(nb_pts)
    for i in range(len(nb_pts)):
        gdlist.append(gdtensor[dim*j:dim*(j+nb_pts[i])])
        gdlist_bg.append(gdtensor[dim*(n+j):dim*(n+j+nb_pts[i])])
        j = j + nb_pts[i]
    return [*gdlist, gdlist_bg]


In [257]:
def test_gradcheck_list2tensor():
    def list2tensor(controls):
        
        controls = controlstensor2list(controls, [nb_pts1], dim)
        
        return controlslist2tensor(controls)
    
    controls = torch.cat([torch.ones(modules.controls[0].shape, requires_grad=True), torch.ones(modules.controls[0].shape, requires_grad=True)])

    return torch.autograd.gradcheck(list2tensor, controls, raise_exception=True)

test_gradcheck_list2tensor()

True

In [86]:
def test_gradcheck_apply_mom():
    def apply_mom(gd, mom, controls, l):

        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        #H.module.fill_controls(controlstensor2list(controls, [nb_pts1, nb_pts2], dim))
        
        H.module.fill_l(l)
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))
        H.module.fill_controls(controlstensor2list(controls, [nb_pts1], dim))
        
        return H.apply_mom()

    modules.compute_geodesic_variables(modules.manifold, constr_Id)
    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)
    controls = controlslist2tensor(modules.controls)

    return torch.autograd.gradcheck(apply_mom, (gd, mom, controls, modules.l), raise_exception=True)

test_gradcheck_apply_mom()

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [  0.0000],
        [-19.4238],
        [  0.0000],
        [-15.7142],
        [-11.4170],
        [ -6.0023],
        [-18.4731],
        [  6.0023],
        [-18.4731],
        [ 15.7142],
        [-11.4170],
        [ 19.4238],
        [  0.0000],
        [ 15.7142],
        [ 11.4170],
        [  6.0023],
        [ 18.4731],
        [ -6.0023],
        [ 18.4731],
        [-15.7142],
        [ 11.4170]])
analytical:tensor([[ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000

In [78]:
def test_gradcheck_innerprod():
    def innerprod(gd, mom, controls, l):
        
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        #H.module.fill_controls(controlstensor2list(controls, [nb_pts1, nb_pts2], dim))
        H.module.fill_l(l)
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))
        H.module.fill_controls(controlstensor2list(controls, [nb_pts1], dim))
        
        
        return sum([H.module.manifold.manifold_list[i].inner_prod_module(H.module.module_list[i]) for i in range(2)])
    
    modules.compute_geodesic_variables(modules.manifold, constr_Id)
    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)
    controls = controlslist2tensor(modules.controls)
    return torch.autograd.gradcheck(innerprod, (gd, mom, controls, modules.l), raise_exception=True)

test_gradcheck_innerprod()

True

In [261]:
def test_gradcheck_geodesic_var():
    def geodesic_var(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))

        H.module.compute_geodesic_variables(H.module.manifold, constr_Id)
        return H.module.l

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)


    return torch.autograd.gradcheck(geodesic_var, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_var()

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000

In [278]:
def test_gradcheck_autoaction():
    def autoaction(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))

        H.module.module_list[1].compute_geodesic_control_from_self()
        return controlslist2tensor(H.module.controls[1])

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)


    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)


test_gradcheck_autoaction()

True

In [277]:
def test_gradcheck_geodesic_controls():
    def geodesic_controls(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))

        H.geodesic_controls()
        return controlslist2tensor(H.module.controls)

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)
    
    def geodesic_controls_gd(p):
        #p = controlslist2tensor(modules.manifold.cotan)
        q = gdlist2tensor(modules.manifold.gd)

        return geodesic_controls(q, p)

    return torch.autograd.gradcheck(geodesic_controls, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_controls()

____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________
____________

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        ...,
        [-204.8001,  -31.0708,  256.2981,  ..., -320.9683,  293.9740,
          293.9740],
        [  25.5605,  546.2762,   22.4517,  ..., -249.1526,  273.0319,
          273.0319],
        [ 118.7862,  111.8729, -151.9882,  ...,  184.5071, -198.3693,
         -198.3693]])
analytical:tensor([[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        ...,
        [ 240.1010,  240.1005, -188.7652,  ..., -323.1905,  295

In [89]:
def test_gradcheck_geodesic_var():
    def geodesic_var(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))

        H.geodesic_controls()
        return H.module.l#controlslist2tensor(H.module.controls)

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)

    return torch.autograd.gradcheck(geodesic_var, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_controls()

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        ...,
        [ 148.0425,  220.4042,  -78.5841,  ..., -436.5642,  399.8480,
          399.8480],
        [ 752.4189,  410.6626, -687.7795,  ..., -338.8842,  371.3637,
          371.3637],
        [-885.1809, -695.5438,  840.9413,  ...,  250.9569, -269.8115,
         -269.8115]])
analytical:tensor([[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
            0.0000],
        ...,
        [ 326.5717,  326.5730, -256.7474,  ..., -439.5868,  402

In [94]:
def test_gradcheck_geodesic_var():
    def geodesic_var(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))

        
        return H.module.compute_geodesic_variables()

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)

    return torch.autograd.gradcheck(geodesic_var, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_controls()

[tensor([[85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648]], grad_fn=<AsStridedBackward>), [tensor([[40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110]], grad_fn=<AddBackward0>)]]
[tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
        1.8090, 0.4122], grad_fn=<SliceBackward>), [tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
       

[tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
        1.8090, 0.4122], grad_fn=<SliceBackward>), [tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
        1.8090, 0.4122], grad_fn=<SliceBackward>)]]
[tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
        1.8090, 0.4122], grad_fn=<SliceBackward>), [tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
        1.8090, 0.4122], grad_fn=<SliceBackward>)]]
____________
[tensor([[85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.264

[tensor([[85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648]], grad_fn=<AsStridedBackward>), [tensor([[40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110]], grad_fn=<AddBackward0>)]]
[tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
        1.8090, 0.4122], grad_fn=<SliceBackward>), [tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
       

[tensor([[85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648]], grad_fn=<AsStridedBackward>), [tensor([[40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110]], grad_fn=<AddBackward0>)]]
[tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
        1.8090, 0.4122], grad_fn=<SliceBackward>), [tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
       

[tensor([[85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648],
        [85.2648, 85.2648]], grad_fn=<AsStridedBackward>), [tensor([[40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110],
        [40.1110, 40.1110]], grad_fn=<AddBackward0>)]]
[tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
        1.8090, 0.4122], grad_fn=<SliceBackward>), [tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
       

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[    0.0000,     0.0000,     0.0000,  ...,     0.0000,     0.0000,
             0.0000],
        [    0.0000,     0.0000,     0.0000,  ...,     0.0000,     0.0000,
             0.0000],
        [    0.0000,     0.0000,     0.0000,  ...,     0.0000,     0.0000,
             0.0000],
        ...,
        [  244.8200,  1051.2932,    76.0786,  ..., -2032.2547,  1861.3364,
          1861.3369],
        [  400.7514,  2010.2192,   -98.7901,  ..., -1577.5438,  1728.7387,
          1728.7391],
        [-1702.6168,   284.5462,  1499.2231,  ...,  1168.2322, -1256.0022,
         -1256.0025]])
analytical:tensor([[    0.0000,     0.0000,     0.0000,  ...,     0.0000,     0.0000,
             0.0000],
        [    0.0000,     0.0000,     0.0000,  ...,     0.0000,     0.0000,
             0.0000],
        [    0.0000,     0.0000,     0.0000,  ...,     0.0000,     0.0000,
             0.0000],
        ...,
        [

In [283]:
def test_gradcheck_geodesic_cont_self():
    def geodesic_cont_self(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        
        trans1.manifold.fill_gd(gd)
        trans1.manifold.fill_cotan(mom)

        trans1.compute_geodesic_control_from_self()
        return controlslist2tensor(trans1.controls)

    gd = modules.manifold.gd[0]
    mom = modules.manifold.cotan[0]

    return torch.autograd.gradcheck(geodesic_cont_self, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_cont_self()

RuntimeError: invalid argument 2: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Call .contiguous() before .view(). at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/TH/generic/THTensor.cpp:213

In [246]:
def test_gradcheck_geo_cont_self():
    def geodesic_geo_cont_self(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
                
        H.module.manifold.manifold_list[j].fill_gd(gd)
        H.module.manifold.manifold_list[j].fill_cotan(mom)
                
        #print(H.module.manifold.gd[j])
        #print(H.module.manifold.cotan[j])

        
        return controlslist2tensor(H.module.controls[1])

    j = 0
    gd = modules.manifold.gd[j]
    mom = modules.manifold.cotan[j]
    
    print(gd, mom)
    
    geodesic_geo_cont_self(gd, mom)
    return torch.autograd.gradcheck(geodesic_geo_cont_self, (gd, mom), raise_exception=True)


test_gradcheck_geo_cont_self()

tensor([2.0000, 1.0000, 1.8090, 1.5878, 1.3090, 1.9511, 0.6910, 1.9511, 0.1910,
        1.5878, 0.0000, 1.0000, 0.1910, 0.4122, 0.6910, 0.0489, 1.3090, 0.0489,
        1.8090, 0.4122], requires_grad=True) tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], grad_fn=<ViewBackward>)


RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 

In [245]:
a=torch.tensor([0.], requires_grad=True)
b=a.clone()
#b.retain_grad()
print(a,b)

tensor([0.], requires_grad=True) tensor([0.], grad_fn=<CloneBackward>)


In [243]:
x = a
x.backward()
y = b
y.backward()
print(a.grad, grad_fn=<CloneBackward>
      b.grad)

tensor([2.]) None


In [244]:
print(a.requires_grad)
print(b.requires_grad)

True
True
